In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import xgboost as xgb



In [44]:
np.random.seed(0)


In [45]:
df = pd.read_csv("dataset/train.csv")

In [46]:
# Encoding categorical variables
categorical_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Status']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Normalizing numerical features
numerical_cols = df.columns.drop(categorical_cols)
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Splitting the dataset
X = df.drop(['Status', 'id'], axis=1)
y = df['Status']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

df.head(), X_train.shape, X_val.shape, X_test.shape

(         id    N_Days  Drug       Age  Sex  Ascites  Hepatomegaly  Spiders   
 0 -1.731832 -0.942430     0  0.858448    1        0             0        0  \
 1 -1.731393  0.497025     1  0.234760    0        0             0        0   
 2 -1.730955  1.277529     1 -1.262634    0        0             1        1   
 3 -1.730517  0.498852     1  0.023603    0        0             0        0   
 4 -1.730079 -1.135271     1 -0.466107    0        0             1        0   
 
    Edema  Bilirubin  Cholesterol   Albumin    Copper  Alk_Phos      SGOT   
 0      0  -0.077237    -0.176908 -0.572940  1.160785 -0.113334  1.336304  \
 1      0  -0.444429     0.068784 -0.024043 -0.275420 -0.197909  0.414968   
 2      2   0.185043    -0.263923  0.004846  0.620561 -0.413812  0.097266   
 3      0  -0.523113    -0.484022 -0.139601 -0.341301 -0.086017 -0.887610   
 4      0  -0.391973    -0.023351  0.293739 -0.275420 -0.333965  0.224347   
 
    Tryglicerides  Platelets  Prothrombin     Stage  Status 

In [47]:
# Logistic Regression
# Setting up the hyperparameter grid
log_reg_params = {
     'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300]
}
model = LogisticRegression(random_state=42)

# Grid search with cross-validation
log_reg_grid = GridSearchCV(model, log_reg_params, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
log_reg_grid.fit(X_train, y_train)

# Best parameters and score for Logistic Regression
log_reg_best_params = log_reg_grid.best_params_
log_reg_best_score = log_reg_grid.best_score_

log_reg_best_params, log_reg_best_score


Fitting 5 folds for each of 270 candidates, totalling 1350 fits


/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

({'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'},
 0.8043401226251928)

In [48]:
best_params = log_reg_grid.best_params_
best_model = log_reg_grid.best_estimator_
best_model.fit(X_val, y_val)


LogisticRegression(C=0.1, random_state=42, solver='newton-cg')

In [49]:
# Decision Tree Classifier
# Setting up the hyperparameter grid
tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
model = DecisionTreeClassifier(random_state=42)

# Grid search with cross-validation
tree_grid = GridSearchCV(model, tree_params, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
tree_grid.fit(X_train, y_train)

# Best parameters and score for Decision Tree
tree_best_params = tree_grid.best_params_
tree_best_score = tree_grid.best_score_

tree_best_params, tree_best_score


Fitting 5 folds for each of 90 candidates, totalling 450 fits


({'criterion': 'gini',
  'max_depth': 10,
  'min_samples_leaf': 4,
  'min_samples_split': 2},
 0.7788293695784592)

In [50]:
best_params = tree_grid.best_params_
best_model = tree_grid.best_estimator_
best_model.fit(X_val, y_val)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=4, random_state=42)

In [51]:
# Adjusting the hyperparameter grid for XGBoost to reduce complexity
xgb_params_reduced = {
    'n_estimators': [100, 200, 400],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.7, 1],
    'random_state': [42]

}

# Grid search with cross-validation (reduced parameter grid)
xgb_grid_reduced = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), 
                                xgb_params_reduced, cv=5, scoring='accuracy')
xgb_grid_reduced.fit(X_train, y_train)

# Best parameters and score for XGBoost (reduced grid)
xgb_best_params_reduced = xgb_grid_reduced.best_params_
xgb_best_score_reduced = xgb_grid_reduced.best_score_

xgb_best_params_reduced, xgb_best_score_reduced

/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/swordfish/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/sklearn.py:1

({'learning_rate': 0.01,
  'max_depth': 7,
  'n_estimators': 400,
  'random_state': 42,
  'subsample': 0.7},
 0.8338597591068547)

In [52]:
from sklearn.metrics import accuracy_score

# Define the models with their best parameters
logistic_regression = LogisticRegression(**log_reg_best_params)  
decision_tree = DecisionTreeClassifier(**tree_best_params)  
xgboost = xgb.XGBClassifier(**xgb_best_params_reduced)  

# Train the models on the training data
logistic_regression.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
xgboost.fit(X_train, y_train)

# Make predictions on the test set
y_pred_logistic = logistic_regression.predict(X_test)
y_pred_decision_tree = decision_tree.predict(X_test)
y_pred_xgboost = xgboost.predict(X_test)

# Calculate accuracy for each model
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracy_decision_tree = accuracy_score(y_test, y_pred_decision_tree)
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)

# Print the accuracies
print(f"Logistic Regression Accuracy: {accuracy_logistic}")
print(f"Decision Tree Accuracy: {accuracy_decision_tree}")
print(f"XGBoost Accuracy: {accuracy_xgboost}")

# Find and print the best model
best_model = None
best_accuracy = 0.0
best_parameters = None

if accuracy_logistic > best_accuracy:
    best_model = "Logistic Regression"
    best_accuracy = accuracy_logistic
    best_parameters = log_reg_best_params

if accuracy_decision_tree > best_accuracy:
    best_model = "Decision Tree"
    best_accuracy = accuracy_decision_tree
    best_parameters = tree_best_params

if accuracy_xgboost > best_accuracy:
    best_model = "XGBoost"
    best_accuracy = accuracy_xgboost
    best_parameters = xgb_best_params_reduced

print(f"The best model is {best_model} with an accuracy of {best_accuracy:.2f}")
print(f"Best Parameters for {best_model}: {best_parameters}")


Logistic Regression Accuracy: 0.7843137254901961
Decision Tree Accuracy: 0.7685009487666035
XGBoost Accuracy: 0.8197343453510436
The best model is XGBoost with an accuracy of 0.82
Best Parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 400, 'random_state': 42, 'subsample': 0.7}


In [53]:
import xgboost as xgb
import pickle

# Define the best hyperparameters
best_params_xgboost = {
    'learning_rate': 0.01,
    'max_depth': 7,
    'n_estimators': 400,
    'random_state': 42,
    'subsample': 0.7
}

# Create and train the XGBoost model with the best parameters
best_xgboost_model = xgb.XGBClassifier(**best_params_xgboost)
best_xgboost_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgboost = best_xgboost_model.predict(X_test)

# Calculate accuracy
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)

# Print accuracy and best model information
print(f"XGBoost Accuracy: {accuracy_xgboost:.2f}")
print(f"The best model is XGBoost with an accuracy of {accuracy_xgboost:.2f}")
print(f"Best Parameters for XGBoost: {best_params_xgboost}")

# Save the model to a file
model_filename = "model_xgb.bin"
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_xgboost_model, model_file)

print(f"Saved the XGBoost model to {model_filename}")


XGBoost Accuracy: 0.82
The best model is XGBoost with an accuracy of 0.82
Best Parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 400, 'random_state': 42, 'subsample': 0.7}
Saved the XGBoost model to model_xgb.bin
